# SyftBox Pipeline Testing Notebook
Test all pipeline approaches with your services

In [1]:
%%capture output --no-stderr

# Install async module
!uv pip install nest_asyncio

In [2]:
# Import the SDK
import syft_hub as sh
import nest_asyncio
import logging
import os

In [3]:
# Import client
from syft_hub import Client
from syft_hub.utils.logger import get_logger

In [4]:
# define utilities
nest_asyncio.apply()
logger = get_logger()
logging.getLogger('syft_hub').setLevel(logging.INFO)
logger.success("Logger initialized successfully!")

2025-09-05 17:04:24 - syft_hub - SUCCESS - Logger initialized successfully!


In [5]:
# Initialize client
client = Client()

2025-09-05 17:04:24 - syft_hub.core.config - INFO - Using config credentials at path: /Users/callisezenwaka/.syftbox/config.json
2025-09-05 17:04:24 - syft_hub.main - INFO - Found existing accounting credentials for callis@openmined.org
2025-09-05 17:04:24 - syft_hub.main - INFO - Client initialized for callis@openmined.org


In [6]:
# Define test message for consistency
test_messages = [{"role": "user", "content": "What is Python programming?"}]

print("=" * 60)
print("PIPELINE TESTING - ALL FLAVORS")
print("=" * 60)

PIPELINE TESTING - ALL FLAVORS


## 1. Inline Approach - Strings Only (Simple)

In [7]:
print("\n1. INLINE APPROACH - Strings Only")
print("-" * 40)

try:
    inline_result = client.pipeline(
        data_sources=["callis@openmined.org/carl-free", "callis@openmined.org/carl-claude"], 
        synthesizer="callis@openmined.org/carl-free"
    ).run(messages=test_messages)
    
    print(f"✅ Success: {inline_result.response.message.content[:100]}...")
    print(f"💰 Cost: ${inline_result.cost}")
    print(f"📄 Search results: {len(inline_result.search_results)}")
    
except Exception as e:
    print(f"❌ Error: {e}")


1. INLINE APPROACH - Strings Only
----------------------------------------
✓ Response received                                                             

✅ Success: PyTorch is a high-level programming language and machine learning library for deep learning in Pytho...
💰 Cost: $0.0
📄 Search results: 3


## 2. Inline Approach - With Parameters (Dicts)

In [8]:
print("\n2. INLINE APPROACH - With Parameters")
print("-" * 40)

try:
    params_result = client.pipeline(
        data_sources=[
            {"name": "callis@openmined.org/carl-free", "topK": 5},
            {"name": "callis@openmined.org/carl-claude", "topK": 8}
        ],
        synthesizer={
            "name": "callis@openmined.org/carl-free", 
            "temperature": 0.8
        }
    ).run(messages=test_messages)
    
    print(f"✅ Success: {params_result.response.message.content[:100]}...")
    print(f"💰 Cost: ${params_result.cost}")
    print(f"📄 Search results: {len(params_result.search_results)}")
    
except Exception as e:
    print(f"❌ Error: {e}")


2. INLINE APPROACH - With Parameters
----------------------------------------
✓ Response received                                                             

✅ Success: Python is a popular high-level dynamic language that can be used to write computer programs. It is a...
💰 Cost: $0.0
📄 Search results: 8


## 3. Object-Oriented Approach - Method-based (Reusable)

In [9]:
print("\n3. OBJECT-ORIENTED APPROACH - Method-based")
print("-" * 40)

try:
    # Create reusable pipeline
    oop_pipeline = client.create_pipeline()
    oop_pipeline.add_source("callis@openmined.org/carl-free", topK=6)
    oop_pipeline.add_source("callis@openmined.org/carl-claude", topK=10)
    oop_pipeline.set_synthesizer("callis@openmined.org/carl-free", temperature=0.7)
    
    # Run multiple times to show reusability
    oop_result1 = oop_pipeline.run(messages=test_messages)
    print(f"✅ First run: {oop_result1.response.message.content[:100]}...")
    print(f"💰 Cost: ${oop_result1.cost}")
    
    oop_result2 = oop_pipeline.run(messages=[{"role": "user", "content": "What is machine learning?"}])
    print(f"✅ Second run: {oop_result2.response.message.content[:100]}...")
    print(f"💰 Cost: ${oop_result2.cost}")
    
except Exception as e:
    print(f"❌ Error: {e}")


3. OBJECT-ORIENTED APPROACH - Method-based
----------------------------------------
                                                                                

2025-09-05 17:08:06 - syft_hub.core.pipeline - ERROR - Search failed for callis@openmined.org/carl-free: RPC_ERROR: Polling timed out after 30/30 attempts
2025-09-05 17:08:06 - syft_hub.core.pipeline - WARNING - Search failed for source callis@openmined.org/carl-free: RPC_ERROR: Polling timed out after 30/30 attempts


✓ Response received

✓ Response received                                                             

✅ First run: PyThon (pronounced "Pythia") is a popular general-purpose scripting language and a software developm...
💰 Cost: $0.0
✓ Response received                                                             

✅ Second run: Machine learning (ML) is a field that employs mathematical algorithms to analyze large amounts of da...
💰 Cost: $0.0


## 4. Mixed Approach - Service Objects + Strings (NEW)

In [10]:
print("\n4. MIXED APPROACH - Service Objects + Strings")
print("-" * 40)

try:
    # Load services as objects
    docs = client.load_service("callis@openmined.org/carl-free")
    carl = client.load_service("callis@openmined.org/carl-claude")
    
    print(f"📄 Loaded service: {docs.name} (supports chat: {docs.supports_chat}, search: {docs.supports_search})")
    print(f"📄 Loaded service: {carl.name} (supports chat: {carl.supports_chat}, search: {carl.supports_search})")
    
    # Mixed pattern - Service objects with strings/dicts
    mixed_result = client.pipeline(
        data_sources=[
            docs,  # Service object
            {"name": "callis@openmined.org/carl-claude", "topK": 5}  # Dict with string
        ],
        synthesizer=docs  # Service object
    ).run(messages=test_messages)
    
    print(f"✅ Success: {mixed_result.response.message.content[:100]}...")
    print(f"💰 Cost: ${mixed_result.cost}")
    print(f"📄 Search results: {len(mixed_result.search_results)}")
    
except Exception as e:
    print(f"❌ Error: {e}")


4. MIXED APPROACH - Service Objects + Strings
----------------------------------------
📄 Loaded service: carl-free (supports chat: True, search: True)
📄 Loaded service: carl-claude (supports chat: True, search: True)
                                                                                

2025-09-05 17:11:43 - syft_hub.core.pipeline - ERROR - Search failed for callis@openmined.org/carl-claude: RPC_ERROR: Polling timed out after 30/30 attempts
2025-09-05 17:11:43 - syft_hub.core.pipeline - WARNING - Search failed for source callis@openmined.org/carl-claude: RPC_ERROR: Polling timed out after 30/30 attempts


✓ Response received

✓ Response received                                                             

✅ Success: Python programming is a programming language that is known for being user-friendly, efficient, and p...
💰 Cost: $0.0
📄 Search results: 3


## 5. Service Object Methods - Direct Usage

In [11]:
print("\n5. SERVICE OBJECT METHODS - Direct Usage")
print("-" * 40)

try:
    # Test individual service methods
    docs = client.load_service("callis@openmined.org/carl-free")
    
    # Direct chat
    if docs.supports_chat:
        chat_result = docs.chat(messages=test_messages)
        print(f"✅ Direct chat: {chat_result.message.content[:100]}...")
        print(f"💰 Cost: ${chat_result.cost}")
    
    # Direct search  
    if docs.supports_search:
        search_result = docs.search(message="Python programming", topK=3)
        print(f"✅ Direct search: Found {len(search_result.results)} results")
        print(f"💰 Cost: ${search_result.cost}")
    
except Exception as e:
    print(f"❌ Error: {e}")

2025-09-05 17:11:49 - syft_hub.main - INFO - Using service: carl-free from datasite: callis@openmined.org



5. SERVICE OBJECT METHODS - Direct Usage
----------------------------------------
                                                                                

2025-09-05 17:11:52 - syft_hub.main - INFO - Using service: carl-free from datasite: callis@openmined.org


✓ Response received

✅ Direct chat: Python programming is a widely used and powerful programming language that is known for its simplici...
💰 Cost: $0
✅ Direct search: Found 3 results
💰 Cost: $0


## 6. Object-Oriented with Service Objects - Method-based

In [ ]:
# ⚠️ Validate that the full_name currently exist
# print("\n6. OBJECT-ORIENTED WITH SERVICE OBJECTS - Method-based")
# print("-" * 40)

# try:
#     # Load services
#     docs = client.load_service("callis@openmined.org/carl-free")
#     carl = client.load_service("callis@openmined.org/carl-claude")
    
#     # Create pipeline using Service objects in method calls
#     # service_pipeline = client.create_pipeline()
#     # service_pipeline.add_source(docs.full_name, topK=7)  # Use full_name
#     # service_pipeline.add_source(carl.full_name, topK=9)  # Use full_name
#     # service_pipeline.set_synthesizer(docs.full_name, temperature=0.6)  # Use full_name
    
#     service_result = service_pipeline.run(messages=test_messages)
#     print(f"✅ Success: {service_result.response.message.content[:100]}...")
#     print(f"💰 Cost: ${service_result.cost}")
#     print(f"📄 Search results: {len(service_result.search_results)}")
    
# except Exception as e:
#     print(f"❌ Error: {e}")

## Comparison Summary

In [12]:
print("\n" + "=" * 60)
print("COMPARISON SUMMARY")
print("=" * 60)

print("""
Approach Comparison:

1. Inline (strings): Simple, one-liners for quick tasks
2. Inline (dicts): Parameters in constructor, still one-liner  
3. Method-based: Full control, reusable pipelines
4. Mixed: Service objects + strings, flexibility
5. Direct service: Individual service operations
6. Service objects + methods: Object-oriented with methods

Recommendation: Use #1 or #2 for simple cases, #3 for complex reusable workflows.
The mixed approach (#4) provides flexibility but adds complexity.
""")

print("\n✅ All tests completed!")


COMPARISON SUMMARY

Approach Comparison:

1. Inline (strings): Simple, one-liners for quick tasks
2. Inline (dicts): Parameters in constructor, still one-liner  
3. Method-based: Full control, reusable pipelines
4. Mixed: Service objects + strings, flexibility
5. Direct service: Individual service operations
6. Service objects + methods: Object-oriented with methods

Recommendation: Use #1 or #2 for simple cases, #3 for complex reusable workflows.
The mixed approach (#4) provides flexibility but adds complexity.


✅ All tests completed!
